In [7]:
from flask import Flask, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
from datetime import datetime, timedelta

# Initialize Flask app
app = Flask(__name__)

# Load the dataset
df = pd.read_csv('water_scarcity.csv')

@app.route('/get_graphs', methods=['POST'])
def get_graphs():
    # Get user inputs from JSON payload
    data = request.get_json()
    state = data.get('state')
    district = data.get('district')

    if not state or not district:
        return jsonify({'error': 'Please provide both state and district'}), 400

    # Filter the dataset by state and district
    filtered_df = df[(df['State'] == state) & (df['District'] == district)]

    # Get the last two months from today's date
    end_date = datetime.today()
    start_date = end_date - timedelta(days=60)  # Last 2 months

    # Convert 'Date' to datetime type
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])

    # Filter data for the last 2 months
    recent_data = filtered_df[filtered_df['Date'] >= start_date]

    if recent_data.empty:
        return jsonify({'error': 'No data found for the given state and district for the last 2 months'}), 404

    # Plot graphs for each parameter
    graphs = {}
    parameters = ['Rainfall (mm)', 'Groundwater Level (m)', 'Temperature (°C)', 'River Water Level (m)']

    for param in parameters:
        plt.figure(figsize=(8, 5))
        plt.plot(recent_data['Date'], recent_data[param], marker='o', linestyle='-', color='b')
        plt.title(f'{param} over Last 2 Months')
        plt.xlabel('Date')
        plt.ylabel(param)
        plt.xticks(rotation=45)
        plt.grid(True)

        # Save plot to a BytesIO object
        img = io.BytesIO()
        plt.savefig(img, format='png')
        img.seek(0)
        img_base64 = base64.b64encode(img.getvalue()).decode('utf-8')

        # Store the base64 string in the dictionary
        graphs[param] = img_base64
        plt.close()

    return jsonify(graphs)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\Manju\AppData\Local\Temp\ipykernel_17244\1309533988.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
127.0.0.1 - - [31/Jan/2025 21:36:06] "POST /get_graphs HTTP/1.1" 200 -


In [9]:
from flask import Flask, request, send_file
import pandas as pd
import matplotlib.pyplot as plt
import io
from datetime import datetime, timedelta

# Initialize Flask app
app = Flask(__name__)

# Load the dataset
df = pd.read_csv('water_scarcity.csv')

@app.route('/get_graphs', methods=['POST'])
def get_graphs():
    # Get user inputs from JSON payload
    data = request.get_json()
    state = data.get('state')
    district = data.get('district')

    if not state or not district:
        return jsonify({'error': 'Please provide both state and district'}), 400

    # Filter the dataset by state and district
    filtered_df = df[(df['State'] == state) & (df['District'] == district)]

    # Get the last two months from today's date
    end_date = datetime.today()
    start_date = end_date - timedelta(days=60)  # Last 2 months

    # Convert 'Date' to datetime type
    filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])

    # Filter data for the last 2 months
    recent_data = filtered_df[filtered_df['Date'] >= start_date]

    if recent_data.empty:
        return jsonify({'error': 'No data found for the given state and district for the last 2 months'}), 404

    # Initialize list to store image files
    img_files = []

    # Plot graphs for each parameter
    parameters = ['Rainfall (mm)', 'Groundwater Level (m)', 'Temperature (°C)', 'River Water Level (m)']
    for param in parameters:
        plt.figure(figsize=(8, 5))
        plt.plot(recent_data['Date'], recent_data[param], marker='o', linestyle='-', color='b')
        plt.title(f'{param} over Last 2 Months')
        plt.xlabel('Date')
        plt.ylabel(param)
        plt.xticks(rotation=45)
        plt.grid(True)

        # Save plot to a BytesIO object
        img = io.BytesIO()
        plt.savefig(img, format='png')
        img.seek(0)
        img_files.append(img)
        plt.close()

    # Return all images as a zip file
    from zipfile import ZipFile
    import os

    # Create a temporary folder to store images
    temp_folder = 'temp_images'
    os.makedirs(temp_folder, exist_ok=True)

    # Save the images to files in the temporary folder
    img_paths = []
    for idx, img in enumerate(img_files):
        img_path = os.path.join(temp_folder, f'graph_{idx+1}.png')
        with open(img_path, 'wb') as f:
            f.write(img.getvalue())
        img_paths.append(img_path)

    # Create a zip file containing the images
    zip_filename = 'graphs.zip'
    with ZipFile(zip_filename, 'w') as zipf:
        for img_path in img_paths:
            zipf.write(img_path, os.path.basename(img_path))

    # Cleanup: remove the temporary image files
    for img_path in img_paths:
        os.remove(img_path)
    os.rmdir(temp_folder)

    # Send the zip file as a response
    return send_file(zip_filename, mimetype='application/zip', as_attachment=True, download_name='graphs.zip')

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Feb/2025 00:06:05] "POST /get_graphs HTTP/1.1" 404 -
C:\Users\Manju\AppData\Local\Temp\ipykernel_17244\3223139913.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
127.0.0.1 - - [01/Feb/2025 00:06:46] "POST /get_graphs HTTP/1.1" 200 -
